In [1]:
""" minimalistic adiabatic parcel model using Numpy, pint, SciPy/odeint and matplotlib """
import math, pint, scipy, chempy, ipywidgets, IPython
from matplotlib import pyplot
si = pint.UnitRegistry()
si.setup_matplotlib()

In [2]:
N_SD = 5 # TODO

IDX_R0 = 0
IDX_RN = N_SD
IDX_Z = N_SD
IDX_S = N_SD + 1
IDX_T = N_SD + 2
IDX_D = N_SD + 3
IDX_ALL = N_SD + 4

units = [None] * IDX_ALL
units[IDX_Z] = si.m
units[IDX_S] = si.dimensionless
units[IDX_T] = si.K
units[IDX_D] = si.kg / si.m**3
units[IDX_R0 : IDX_RN] = [si.m] * N_SD

In [3]:
def sample_spectrum(parameters):
    spectrum = scipy.stats.lognorm(*(
        math.log(parameters['lognormal_geometric_stdev']),
        0,
        parameters['lognormal_median_radius'].to(si.m).magnitude
    ))
    # TODO: plot spectrum.pdf & the sampled distribition
    return {
        'multiplicity': [parameters['total_aerosol_number'] / N_SD] * N_SD,
        'dry radius': [
            spectrum.ppf(
                (2 * i - 1) / (2 * N_SD)
            ) * si.m
            for i in range(1, N_SD + 1)
        ]
    }

CONST = {
    'M_dry': (
        0.76 * chempy.Substance.from_formula("N2").mass * si.gram / si.mole +
        0.23 * chempy.Substance.from_formula("O2").mass * si.gram / si.mole +
        0.01 * chempy.Substance.from_formula("Ar").mass * si.gram / si.mole
    ),
    'M_vap': chempy.Substance.from_formula("H2O").mass * si.gram / si.mole,
    'g': scipy.constants.g * si.m / si.s**2,
    'water_density': 1000 * si.kg / si.m**3,
    "specific_heat_constant_pressure": 1005 * si.J / si.kg / si.K,
    "latent_heat": 2.27e6 * si.J / si.kg,
    "surface_tension_water": 7.5e-2 * si.N / si.m,
}
CONST['R_dry'] = scipy.constants.R * si.J / si.mole / si.K / CONST['M_dry']
CONST['eps'] = CONST['M_vap'] / CONST['M_dry']

In [4]:
def funct(state, _, parameters):
    """ returns time derivative of the state vector """
    saturated = state[IDX_S]
    temperature = state[IDX_T] * units[IDX_T]
    air_density = state[IDX_D] * units[IDX_D]

    condensation_rate_constant = CONST['water_density'] * 4 * math.pi / parameters['total_aerosol_number'] / parameters['mass_of_dry_air']
    pressure = 938.5 * si.hPa
    
    water_vapor_diffusivity = 1e-5 * si.m**2 / si.s * (
            (0.015 / si.K * temperature) - 1.9)
    thermal_conductivity = (
            (1.5e-11 * si.W / si.m / si.K**4) * temperature**3
            + (-4.8e-8 * si.W / si.m / si.K**3) * temperature**2
            + (1e-4 * si.W / si.m / si.K**2) * temperature
            + -3.9e-4 * si.W / si.m / si.K
        )

    saturated_vapor_pressure = 6.112 * si.hPa * math.exp(
        (17.67 * si.dimensionless * temperature) / (temperature + 243.5 * si.K))

    a_coeff = (CONST['g'] / (CONST['R_dry'] * temperature)  * (
        (CONST['eps'] * CONST["latent_heat"]) / (CONST["specific_heat_constant_pressure"] * temperature) - 1 )
    )

    b_coeff = air_density * (
        (CONST['R_dry'] * temperature / (CONST['eps'] * saturated_vapor_pressure))
        +
        (CONST['eps'] * CONST["latent_heat"] ** 2 / (pressure * temperature * CONST["specific_heat_constant_pressure"]))
    )

    drop_surface_term = (2 * CONST['surface_tension_water']) / (CONST['R_dry'] * CONST['water_density'] * temperature)

    a_value = ((saturated_vapor_pressure * water_vapor_diffusivity)
               / (CONST['water_density'] * temperature * CONST['R_dry']))
    b_value = (((thermal_conductivity * temperature)/ (CONST['water_density'] * CONST["latent_heat"]))
               * ((CONST["latent_heat"] / (CONST['R_dry'] * temperature)) - 1))

    g_term = a_value + b_value

    deriv = [None] * len(state)
    deriv[IDX_Z] = parameters['updraft']

    condensation_rate = 0 / si.s

    for i in range(IDX_R0, IDX_RN):
        radii = state[i]* units[i]
        drop_solute_term = parameters['kappa'] * (parameters['sampled_spectrum']['dry radius'][i])**3
        deriv[i] = ((g_term/radii) *
                    (saturated - 1 - (drop_surface_term/radii)
                     +
                     (drop_solute_term/(radii**3))))

        condensation_rate += (condensation_rate_constant *
                              (parameters['sampled_spectrum']['multiplicity'][i] * radii**2 * deriv[i])
                             )
    deriv[IDX_S] = a_coeff * parameters['updraft']  + b_coeff * condensation_rate
    deriv[IDX_T] = - ((CONST["latent_heat"] / CONST["specific_heat_constant_pressure"])
                    * condensation_rate) - ((CONST['g'] / CONST["specific_heat_constant_pressure"])
                    * (deriv[IDX_Z]))
    deriv[IDX_D] = (- ((air_density * CONST['g'] * deriv[IDX_Z]) / (CONST['R_dry'] * temperature))
                    - ((air_density / temperature) * deriv[IDX_T]))
    return [
        deriv[i].to(units[i] / si.s).magnitude
        for i in range(len(units))
    ]

In [8]:
def integrate(parameters):    
    initial_condition = [None] * IDX_ALL
    initial_condition[IDX_S] = 0.8 * si.dimensionless
    initial_condition[IDX_Z] = 0 * si.m
    initial_condition[IDX_R0 : IDX_RN] = [1 * si.um] * N_SD
    initial_condition[IDX_T] = 284.3 * si.K
    initial_condition[IDX_D] = 1.293 * si.kg / si.m**3
    
    nt = 40
    dt_out = 5 * si.s
    
    integration_result = scipy.integrate.odeint(
        funct,
        [x.to(units[i]).magnitude for i, x in enumerate(initial_condition)],
        t=[(i * dt_out).to(si.s).magnitude for i in range(nt + 1)],
        args=(parameters,)
    )
    return [
        integration_result[:, i] * units[i]
        for i in range(integration_result.shape[1])
    ]

In [16]:
def plot(integration_result_with_units):
    PLOTS = (
        ((IDX_S,), "saturation", None),
        (range(IDX_R0, IDX_RN), "drop radii", si.um),
        ((IDX_T,), "temperature", None),
        ((IDX_D,), "density", None),
    )
    
    fig, axs = pyplot.subplots(1, len(PLOTS), sharey=True)
    
    for i, (vars, name, unit) in enumerate(PLOTS):
        for idx in vars:
            axs[i].plot(
                integration_result_with_units[idx],
                integration_result_with_units[IDX_Z],
            )
        axs[i].set_title(f"{name} vs. height")
        if unit: axs[i].xaxis.set_units(unit)
    
    for ax in axs:
        ax.grid()

In [18]:
output = ipywidgets.Output()

def parcel(kappa, updraft_m_s):
    with output:
        IPython.display.clear_output(wait=True)
        
        parameters = {
            'updraft':  updraft_m_s * si.m / si.s,
            'kappa': kappa,
            'total_aerosol_number': 1e6,
            'lognormal_geometric_stdev': 1.2,
            'lognormal_median_radius': .05 * si.um,
            'mass_of_dry_air': 1 * si.kg,
        }
                
        parameters['sampled_spectrum'] = sample_spectrum(parameters)
        integration_result_with_units = integrate(parameters)
        plot(integration_result_with_units)
        
        pyplot.show()

ipywidgets.interact(parcel, kappa=1.28, updraft_m_s=2)
IPython.display.display(output)

# TODO: plot spectrum (fig 1 in Jensen & Nugent): analytic curve + sampled spectrum

interactive(children=(FloatSlider(value=1.28, description='kappa', max=3.84, min=-1.28), IntSlider(value=2, de…

Output()